# Full text search on Twitter

The parameters in the cell below can be adjusted to search for other texts and time frames. 

### How to further explore the text?

 You an use the symbols `+` (i.e. *AND*), `|` (i.e. *OR*), `-`  (*NOT*), `()` (to aggregate operators) and `""` (to search for consecutive tokens). For example, `"deutschland ist" + (afd | csu) -die` would search for all texts containing (1) *deutschland ist*, (2) either *afd* or *csu*, but not  *die*.

***Alternatively***, you can direcly use the [politicians API](http://mediamonitoring.gesis.org/api/politicians/swagger/) or [organizations API](http://mediamonitoring.gesis.org/api/organizations/swagger/), or access them with the [SMM Wrapper](https://pypi.org/project/smm-wrapper/).

## A. Set Up parameters

In [ ]:
# Parameters: 
search_text = 'steuer'
from_date = '2017-09-01'
to_date = '2018-12-31'
aggregation = 'week'

## B. Using the SMM API (Politicians and Organizations)

In [ ]:
library("httr")
library("jsonlite")

#prepare URLs
base <- "http://mediamonitoring.gesis.org/api/"
url_politicians <- paste(base,"politicians/twitter/tweets_by/politicians/?", sep="")
url_organizations <- paste(base,"organizations/twitter/tweets_by/organizations/?", sep="")
url_general <- paste(base,"organizations/twitter/general_population/?", sep="")

#prepare parameters
params = list(
    text_contains=search_text,
    from_date=from_date,
    to_date=to_date,
    aggregate_by=aggregation
)

# using the api to get the data
politicians = as.data.frame(fromJSON(content(GET(url_politicians, query=params), "text", encoding="UTF-8"), flatten = TRUE))
organizations = as.data.frame(fromJSON(content(GET(url_organizations, query=params), "text", encoding="UTF-8"), flatten = TRUE))
general = as.data.frame(fromJSON(content(GET(url_general, query=params), "text", encoding="UTF-8"), flatten = TRUE))

# merge the data
merged <- Reduce(function(x,y) merge(x = x, y = y, by = "labels"), 
       list(politicians, organizations, general))

## C. Plotting

In [ ]:
library("ggplot2")
options(repr.plot.width=8, repr.plot.height=4)

ggplot(data = merged, mapping = aes(as.Date(labels))) +
  geom_line(aes(y = values.x, color="Politicians", group=response_type.x)) +
  geom_line(aes(y = values.y, color="Organizations", group=response_type.y)) +
  geom_line(aes(y = values, color="General political tweets", group=response_type)) +
  labs(title = "Twitter (Tweets)", y = "Tweets") +
  theme(axis.text.x = element_text(angle = 60, hjust = 0.5, vjust = 0.5),
        axis.title.x = element_blank(), legend.title = element_blank()) +
scale_x_date(date_breaks = "1 month")